In [ ]:
import os
import time
import google.genai as genai
from google.genai import types
import time
os.environ["GEMINI_API_KEY"] = "YOUR-GEMINI-KEY-HERE"

In [ ]:
from google.colab import files

uploaded = files.upload()  # This will open a file picker dialog


Saving videotest.mp4 to videotest.mp4


----------------------------------------------------------------------------------------------


I was testing the functionality of the new 2.5 pro version of gemini: https://developers.googleblog.com/en/gemini-2-5-pro-io-improved-coding-performance/ but it seems that some features are not working as described in the documentation and low resolution is only available in gemini-2.0-flash-001** (Note: using gemini-2.5-pro-preview-05-06 shows the following error: "Received empty response text from model". )

In [ ]:

api_key = os.environ.get("GEMINI_API_KEY")
if not api_key:
    raise ValueError("Please set the GEMINI_API_KEY environment variable.")

client = genai.Client(api_key=api_key)
video_path = "videotest.mp4"

def wait_for_file_active(client, file_name, timeout=60, interval=5):
    elapsed = 0
    while elapsed < timeout:
        file_info = client.files.get(name=file_name)
        print(f"File state: {file_info.state}")
        if file_info.state.name == "ACTIVE":
            return file_info
        elif file_info.state.name == "FAILED":
            raise RuntimeError("File processing failed.")
        time.sleep(interval)
        elapsed += interval
    raise TimeoutError(f"File did not become ACTIVE within {timeout} seconds.")

uploaded_file = client.files.upload(file=video_path)
print(f"Uploaded file: {uploaded_file.name}, initial state: {uploaded_file.state}")

file_info = wait_for_file_active(client, uploaded_file.name)
print(f"File is now ACTIVE: {file_info.state}")

file_part = types.Part(
    file_data=types.FileData(
        file_uri=file_info.uri  # Use full URI here
    )
)

contents = [
    types.Content(
        parts=[
            file_part,
            types.Part(text="Describe with full details the video uploaded")
        ]
    )
]

generate_content_config = types.GenerateContentConfig(
    temperature=0.7,
    max_output_tokens=512,
    media_resolution=types.MediaResolution.MEDIA_RESOLUTION_LOW,  # gemini-2.5-pro-preview-05-06 is not working with resolution low even if the documentation say so
)

response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents=contents,
    config=generate_content_config,
)

text_output = "".join(
    part.text for part in response.candidates[0].content.parts if part and part.text
)

if text_output.strip():
    print("Model response:\n", text_output)
else:
    print("Received empty response text from model.")


Uploaded file: files/fbczsofhgls7, initial state: FileState.PROCESSING
File state: FileState.PROCESSING
File state: FileState.ACTIVE
File is now ACTIVE: FileState.ACTIVE
Model response:
 Here's a detailed description of the video:

**Visuals:**

The video shows a scuba diver underwater, close to a coral reef. The water is clear and blue, providing good visibility. The diver is wearing a black and red wetsuit, a mask, and yellow fins. They are equipped with scuba gear, including a tank and regulator. 

The diver is positioned near a large coral formation, which is covered in various textures and colors. A large school of small, silver-colored fish is clustered near the coral, creating a dense, shimmering mass.

The diver is mostly stationary, hovering in the water with their arms extended in front of them. They appear to be observing and perhaps filming the school of fish. Bubbles from the diver's breathing apparatus float upwards towards the surface.

The seabed is visible in the lower